# AWS Lambda function modified by IAM user

Detect when an IAM user modifies an AWS Lambda function, which could indicate an attempt to maintain persistence or exfiltrate data in an AWS environment.

## Detection Query
Execute the following query to find alerts related to this rule.

In [ ]:
%ingest.source_type:"aws:cloudtrail"
eventSource:lambda.amazonaws.com
eventName:UpdateFunctionCode*
userIdentity.type:IAMUser
(not userIdentity.invokedBy:(cloudformation.amazonaws.com "AWS Internal"))
| stats countdistinct(requestParameters.functionName) as num_distinct_values by userIdentity.arn
| where num_distinct_values > 0

## Investigation Steps
1. **Identify the Principal**: Who performed the action?
2. **Review Context**: What else did this principal do?
3. **Check Permissions**: Did the principal have legitimate reasons for this action?
4. **Verify Resource**: Is the resource critical or sensitive?

In [ ]:
%ingest.source_type:aws:cloudtrail
userIdentity.arn: <PRINCIPAL_ARN>
| count(eventName) by eventName